# **Basic GNN Model Hyperparameter Optimization**

This notebook search for the best hyperparameters by implementing a Cross-Validation Setup. The Hyperparameters that will be searched are the Learning Rate, the Weight Decay and the Embedding Size of the Graph Neural Network.
An Early Stopping Mechanism will help terminate trials that are not improving.

The data from the UPFD Framework has been already split in Training, Validation and Test Set and is downloadable by using simple commands inside the Pytorch Geometric environment.

## Download and import Libraries for the Environment 


---



Download the right libraries depending if we are using a CPU or a GPU.

In [8]:

import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


#if torch.cuda.is_available():
  #!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu102.html
#else:
  #!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cpu.html

#!pip install optuna
import optuna


from torch.nn import Linear, LogSoftmax
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, global_max_pool

from tool_box.upfd_dataset import ext_UPFD
from tool_box.GNN_train import optimize_GNN, train_step, val_step

# Set GPU as Device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(f"Training Device: {device}")


Training Device: cpu


### Import data

In [2]:
path = './data/'

In [3]:
train_dataset = ext_UPFD(name = 'original', root = path, n_features=[], g_features=[], split = 'train')
val_dataset = ext_UPFD(name = 'original', root = path, n_features=[], g_features=[], split='val')
test_dataset = ext_UPFD(name = 'original', root = path, n_features=[], g_features=[], split='test')

# Training Functions

In [4]:
class GNN(torch.nn.Module):

    def __init__(self, num_n_feature, num_g_feature, emb_size):
        super(GNN, self).__init__()
        torch.manual_seed(42)
        self.emb_size = emb_size

        self.in_layer = GCNConv(num_n_feature, emb_size)
        self.conv1 = GCNConv(emb_size, emb_size)
       
       # check if we have graph features to concatenate or not
        i = 2
        if num_g_feature:
             self.lin_g = Linear(num_g_feature, emb_size)
             i = 3

        self.out_layer = Linear(i * emb_size, 2)
        self.act = LogSoftmax(dim=-1)
    
    def forward(self, x, edges_idx, batch_idx, g_features):
        #pdb.set_trace()
        x = self.in_layer(x, edges_idx)

        x = self.conv1(x, edges_idx)

        flatten = torch.cat([global_mean_pool(x, batch_idx),
                             global_max_pool(x, batch_idx)], axis=1)

        if g_features.size()[-1]:
            g_ft = self.lin_g(g_features)
            flatten = torch.cat([flatten, g_ft], axis=1)

        out = self.act(self.out_layer(flatten))

        return out

# **Training**

Training Function that will be executed every time a trial is done. In this function the hyperparameters are chosen based on suggestions from the hyperparameter optimizer chosen with Optuna.


In [5]:
epochs_max = 60     

In [6]:
study = optimize_GNN(GNN, train_dataset, val_dataset, num_trials = 40)

[I 2021-12-31 16:28:00,187] A new study created in memory with name: no-name-ad52a69e-f599-4e8c-b23d-388d9e8c5c4c
[I 2021-12-31 16:31:07,989] Trial 0 finished with value: 0.8728170955882353 and parameters: {'learning_rate': 0.005, 'weight_decay': 0.001, 'batch_size': 512, 'embedding_space_dim': 160}. Best is trial 0 with value: 0.8728170955882353.
[I 2021-12-31 16:31:50,217] Trial 1 finished with value: 0.8148488562091504 and parameters: {'learning_rate': 0.01, 'weight_decay': 0.01, 'batch_size': 64, 'embedding_space_dim': 40}. Best is trial 0 with value: 0.8728170955882353.
[I 2021-12-31 16:34:08,223] Trial 2 finished with value: 0.8434627757352942 and parameters: {'learning_rate': 0.001, 'weight_decay': 0.005, 'batch_size': 512, 'embedding_space_dim': 140}. Best is trial 0 with value: 0.8728170955882353.
[I 2021-12-31 16:35:57,566] Trial 3 finished with value: 0.8742851307189542 and parameters: {'learning_rate': 0.005, 'weight_decay': 0.001, 'batch_size': 64, 'embedding_space_dim': 1

After the training, we pick the model that performed best on the validation set and test it on the Test Set. Obviously no hyperparameters have been chosen by looking on the Test Set.

In [11]:
pruned_trials = study.get_trials(deepcopy = False, states = [optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy = False, states = [optuna.trial.TrialState.COMPLETE])

# Extract the best hyperparameters 
best_parameters = study.best_params
embedding_space_dim = best_parameters["embedding_space_dim"]
learning_rate = best_parameters["learning_rate"]
weight_decay = best_parameters["weight_decay"]
batch_size = best_parameters["batch_size"]

# Initialize Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize Model with best Hyperparameters
num_node_features = train_dataset.num_features
num_graph_features = train_dataset.g_features
best_model = GNN(num_n_feature = num_node_features, num_g_feature = num_graph_features, emb_size = embedding_space_dim).to(device)
best_optimizer = torch.optim.Adam(best_model.parameters(), lr = learning_rate, weight_decay = weight_decay)
loss_f = torch.nn.NLLLoss()

# Train the Final Model
acc_losses_train, acc_losses_val, acc_losses_test = [], [], []

for epoch in range(epochs_max):
  loss_train, acc_train = train_step(best_model, train_loader, best_optimizer, loss_f)
  loss_val, acc_val = val_step(best_model, val_loader, loss_f)
  loss_test, acc_test = val_step(best_model, test_loader, loss_f)
        
  acc_losses_train.append([loss_train, acc_train])
  acc_losses_val.append([loss_val, acc_val])
  acc_losses_test.append([loss_test, acc_test])


print(f"BEST HYPERPARAMETERS:")
print(f"Embedding Space Dimension = {embedding_space_dim}")
print(f"Learning Rate = {learning_rate}")
print(f"Weight Decay = {weight_decay}")
print(f"Batch Size = {batch_size}")

print(f"Best model final Test loss: {acc_losses_test[-1][1]}")
print(f"Best model final Test accuracy: {acc_losses_test[-1][0]}")
    


BEST HYPERPARAMETERS:
Embedding Space Dimension = 140
Learning Rate = 0.01
Weight Decay = 0.001
Batch Size = 256
Best model final Test loss: 0.8877044593663912
Best model final Test accuracy: 0.30249659419059755


# Plots